In [ ]:
(ql:quickload :kenzo)

In [ ]:
(in-package "CAT")

In [ ]:
(cat-init)

# Objects with effective homology

## Introduction

The present chapter describes the set of programming tools for handling *objects with effective homology*. The theoretical material may be found in the paper *Constructive Algebraic Topology* by Julio Rubio Garcia and Francis Sergeraert (Available at the web site
[**https://www-fourier.ujf.grenoble.fr/~sergerar/**](https://www-fourier.ujf.grenoble.fr/~sergerar/)). The terminology used in this chapter is compatible with this reference.). The terminology used in this chapter is compatible with this reference.

## Reduction

A *reduction* in ``Kenzo`` is a 5-tuple $(\hat C, C, f, g, h)$:

$$
\begin{align}
\hat{C}\quad & \qquad\stackrel{h}{\longrightarrow} & \mbox{$^s\hat{C}$} \\
{\scriptstyle f} \downarrow \uparrow {\scriptstyle g} & & \\
C\quad & &
\end{align}
$$

where $\hat C$ and $C$ are chain complexes, $f$ and $g$  chain complex morphisms and
$h$ a homotopy operator. Hereafter, $\hat C$ is called the *top chain complex* and $C$ the *bottom chain complex*. ${}^s\hat C$ is $\hat C$ shifted, i.e. $h$ has degree $1$. The mappings $f$, $g$, $h$, together with the differential operator $d$ on $\hat C$, must verify the following relations:

$$
\begin{eqnarray*}
f \circ g & = & 1_C \\
h \circ d + d \circ h & = & 1_{\hat C} - g \circ f \\
f \circ h & = & 0   \\
h \circ g & = & 0   \\
h \circ h & = & 0
\end{eqnarray*}
$$

The morphisms $f$ and $g$ and the homotopy operator $h$ describe the (big) chain complex $\hat C$ as the direct sum

$$ \hat C= \hat C_1 \oplus \hat C_2$$

where $\hat C_1 = Im(g) \simeq C$ and  $\hat C_2= Ker(f)$ ($\hat C_2$ is acyclic).

### Representation of a reduction

A reduction\index{reduction!representation} is implemented as an instance of the ``CLOS`` class ``REDUCTION``, whose definition is:

```lisp
(DEFCLASS REDUCTION ()
     ;; Top Chain Complex
    ((tcc :type chain-complex :initarg :tcc :reader tcc1)
     ;; Bottom Chain Complex
     (bcc :type chain-complex :initarg :bcc :reader bcc1)
     (f :type morphism :initarg :f :reader f1)
     (g :type morphism :initarg :g :reader g1)
     (h :type morphism :initarg :h :reader h1)
     ;; IDentification NuMber
     (idnm :type fixnum :initform (incf *idnm-counter*) :reader idnm)
     ;; ORiGiN
     (orgn :type list :initarg :orgn :reader orgn)))
```

This class has 7 slots:

1. ``tcc``, the object of type ``chain-complex`` representing the chain complex $\hat C$ (top chain complex).
2. ``bcc``,  the object of type ``chain-complex`` representing the chain complex $C$ (bottom chain complex).
3. ``f``, the object of type ``morphism``representing the morphism $f$.
4. ``g``, the object of type ``morphism`` representing the morphism $g$.
5. ``h``, the object of type ``morphism``  representing the morphism $h$.
6. ``idmn``, an integer, number plate for the object.
7. ``orgn``, a comment list carefully chosen.

The accessors of the slots are the functions whose name appears after the specifier ``:reader`` in the class definition. A printing method has been associated to the class ``REDUCTION`` and the external representation of an instance is a string like ``[Kn Reduction]``, where ``n`` is the number plate of the ``Kenzo`` object.

### The function ``build-rdct``

To facilitate the construction of instances of the ``REDUCTION`` class, the
software provides the function ``build-rdct``.

```lisp
build-rdct :f f :g g :h h :orgn orgn
```

defined with keyword parameters. The returned value is an instance of the class ``REDUCTION``. 
The keyword arguments are:

- *f*, the object of type ``morphism`` representing the morphism $f$.
- *g*, the object of type ``morphism`` representing the morphism $g$.
- *h*, the object of type ``morphism`` representing the morphism $h$.
- *orgn*, the comment list carefully chosen since the system does not build a new instance of the class if it finds in the list of already built reductions, ``*rdct-list*``, a reduction  with the same comment list.

The ``tcc`` slot and the ``bcc`` slot of the instance are taken respectively from the ``sorc`` slot (source slot) and the ``trgt`` slot (target slot) of the morphism $f$. The function ``build-rdc`` controls the validity of the degrees of the morphisms and pushes the new created instance on the list ``*rdct-list*``.

### Useful macros and functions

``(cat-init)`` *[Function]*
> Clear among others, the list ``*rdct-list*``, list of user created reductions  and reset the global counter to 1. See also the description of this function in chapter 1.

``(rdct n)`` *[Function]*
> Retrieve in the list ``*rdct-list*`` the reduction instance whose identification (as ``Kenzo`` object) is ``n``. If it does not exist, return ``NIL``.

``(bcc rdct &rest args)`` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the bottom chain complex of the reduction. Otherwise, it applies  the differential of the bottom chain complex of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

``(tcc rdct &rest args)`` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the top chain complex of the reduction. Otherwise, it applies  the differential of the top chain complex of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

``(f rdct &rest args)`` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the morphism $f$ of the reduction. Otherwise, it applies the morphism $f$ of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

``(g rdct &rest args)`` *[Macro]*
> With only one argument (a reduction *rdct* this macro selects the morphism $g$ of the reduction. Otherwise, it applies  the morphism $g$ of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

``(h rdct &rest args)`` *[Macro]*
> With only one argument (a reduction *rdct*) this macro selects the morphism $h$ of the reduction. Otherwise, it applies the morphism $h$ of the reduction *rdc* on the arguments *args*, (either *degree generator* or *cmb*).

``(trivial-rdct chcm)`` *[Function]*
> Build the trivial reduction involving only the chain complex chcm, as shown in the following diagram,
$$
\begin{align}
 C \quad & \quad\stackrel{Zero} {\longrightarrow} & {}^s C \\
 {\scriptstyle {Id}} \downarrow \uparrow {\scriptstyle {Id}} & &  \\
  C \quad & &
\end{align}
$$
where the morphism *Zero* is the zero morphism of degree 1 in the chain complex $C$ and *Id* is the identity morphism in that chain complex (see the functions ``zero-mrph`` and ``idnt-mrph`` in the chain complex chapter).

``(cmps brdct trdct)`` *[Method]*
> Build a new reduction from the two reductions $r1$ and $r2$ (here, respectively *brdct* and *trdct*). This is done by a call to ``build-rdct`` with the following parameters:
$$
\begin{eqnarray*}
f & = & f_{r1}\circ f_{r2}, \\
g & = & g_{r2}\circ g_{r1}, \\
h & = &  h_{r2}  + g_{r2} \circ h_{r1} \circ f_{r2}.
\end{eqnarray*}
$$
The compositions and additions of morphisms are realized respectively by the methods ``cmps``, ``i-cmps`` and ``add`` (see chapter 1). We recall that the ``tcc`` and the ``bcc`` slots of this new created reduction are respectively the source and target slots of the new morphim $f$.

### Verification functions

The two following functions are very helpful to verify the coherence of the the various mappings involved in a reduction. Let us recall the diagram of a reduction:
$$
\begin{align}
\hat{C} \quad & \quad\stackrel{h}{\longrightarrow} & \mbox{$^s\hat{C}$} \\
{\scriptstyle f} \downarrow \uparrow {\scriptstyle g} & & \\
C \quad & &
\end{align}
$$

``(pre-check-rdct rdct)`` *[Function]*
> Assign to the following lisp global variables, the morphism instances computed from the morphisms of the reduction *rdct*, according to the  formulas:
$$
\begin{align}
  \mathtt{*tdd*} & = & d_{\hat C} \circ d_{\hat C},\\
  \mathtt{*bdd*} & = & d_C \circ d_C , \\
  \mathtt{*id-fg*} & = &  f \circ g - Id_C,
  \mathtt{*id-gf-dh-hd*} & = & Id_{\hat C} - g \circ f - (d_{\hat C} \circ h + h \circ d_{\hat C}),\\
  \mathtt{*hh*} & = & h \circ h, \\
  \mathtt{*fh*} & = & f \circ h, \\
  \mathtt{*hg*} & = & h \circ g, \\
  \mathtt{*df-fd*} & = & d_C \circ f - f \circ d_{\hat C}, \\
  \mathtt{*dg-gd*} & = & d_{\hat C} \circ g - g \circ d_C.
\end{align}
$$
In these formulas  $d_C$ and $d_{\hat C}$ are the respective boundary operators of the
chain complexes $C$ and $\hat C$. $Id_C$ and $Id_{\hat C}$ are the identity morphisms
on the chain complexes $C$ and $\hat C$  built by the function ``idnt-mrph``

``(check-rdct)`` *[Function]*
> Map  all the morphisms prepared by the function *pre-check-rdct* on chosen combinations.
This function having no parameters, the user must  assign to  the two lisp global variables:
``*tc*`` and ``*bc*``,  valid combinations belonging respectively to $\hat C$ and $C$. The call to this function is simply ``(check-rdct)``. This function knows what morphism to apply
to the combinations and pauses after each evaluation to allow the user to inspect each result.
To resume the execution, the user must enter a blank character. If the morphisms are coherent, the result of each mapping is a null combination.

#### Example

In [ ]:
(defun cdelta (dmns)
    (build-chcm
     :cmpr #'l-cmpr
     :basis :locally-effective
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (make-cmbn
             :degr (1- degr)
             :list (do ((rslt +empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(locally effective version of C_* delta ,dmns)))

In [ ]:
(defun make-f (tdmns bdmns)
    (build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta bdmns) :degr 0
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (< pos degr)
                             (zero-cmbn degr)
                             (cmbn degr 1 (nconc (butlast gmsm) (list bdmns))))
                         (cmbn degr 1 gmsm))))
     :strt :gnrt
     :orgn `(projection delta ,tdmns => delta ,bdmns)))

In [ ]:
(defun make-g (tdmns bdmns)
    (build-mrph
     :sorc (cdelta bdmns) :trgt (cdelta tdmns) :degr 0
     :intr #'identity
     :strt :cmbn
     :orgn `(injection delta ,bdmns => delta ,tdmns)))

In [ ]:
(defun make-h (tdmns bdmns)
    (build-mrph
     :sorc (cdelta tdmns) :trgt (cdelta tdmns) :degr +1
     :intr #'(lambda (degr gmsm)
                (let ((pos (position-if #'(lambda (vertex) (>= vertex bdmns))
                                        gmsm)))
                     (if pos
                         (if (member bdmns gmsm)
                             (zero-cmbn (1+ degr))
                             (cmbn (1+ degr) (-1-expt-n pos)
                                       (append (subseq gmsm 0 pos) (list bdmns)
                                               (subseq gmsm pos))))
                         (zero-cmbn (1+ degr)))))
     :strt :gnrt
     :orgn `(homotopy for delta ,tdmns => ,bdmns)))

In [ ]:
(defun make-rdct (tdmns bdmns)
    (let ((rdct (build-rdct
                 :f (make-f tdmns bdmns)
                 :g (make-g tdmns bdmns)
                 :h (make-h tdmns bdmns)
                 :orgn `(reduction delta ,tdmns ,bdmns))))
         rdct))

In [ ]:
(def rdct (make-rdct 6 3))

In [ ]:
(orgn rdct)

In [ ]:
(pre-check-rdct rdct)

In [ ]:
(setf *tc* (cmbn 2 1 '(0 1 2) 10 '(1 2 3) 100 '(1 2 4) 1000 '(2 3 4)))

In [ ]:
(setf *bc* (cmbn 3 4 '(0 1 2 3)))

In [ ]:
(check-rdct-no-wait)

In [ ]:
(def trdct (make-rdct 6 4))

In [ ]:
(def brdct (make-rdct 4 3))

In [ ]:
(setf rdct (cmps brdct trdct))

In [ ]:
(pre-check-rdct rdct)

In [ ]:
(check-rdct-no-wait)

### 2.4.1 Useful functions related to the perturbation lemma
#### Example

In [ ]:
(def hmeq (trivial-hmeq (cdelta 4)))

In [ ]:
(setf hmeq (add hmeq (opps (dffr (cdelta 4)))))

In [ ]:
(gnrt-? (dffr (rbcc hmeq)) 3 '(0 1 2 3))

### 2.5.1 Representation of a combination in a bicone

In [ ]:
(def comb-bic (cmbn 3 2 (bcnb 'b1) 4 (bcnb 'b1) 6 (bcnb 'b3)
                    3 (bcnb 'c1) 5 (bcnb 'c1) 7 (bcnb 'd1)))

In [ ]:
(cmbn-list comb-bic)

### 2.5.2 Useful functions and macros for bicones
#### Examples

In [ ]:
(def comb-b (cmbn 3 2 'b1 4 'b2 6 'b3))

In [ ]:
(def comb-c (cmbn 4 3 'c1 5 'c2))

In [ ]:
(def comb-d (cmbn 3 7 'd1))

In [ ]:
(def comb-bic (make-bicn-cmbn comb-b comb-c comb-d))

In [ ]:
(bicn-cmbn-cmbnb comb-bic)

In [ ]:
(bicn-cmbn-cmbnc comb-bic)

In [ ]:
(bicn-cmbn-cmbnd comb-bic)

In [ ]:
(multiple-value-bind (b c d)
    (dispatch-bicn-cmbn comb-bic)
    (list b c d))

## 2.6 Construction of a bicone from 2 reductions
#### Example

In [ ]:
(cat-init)

In [ ]:
(defun cdelta (dmns)
    (build-chcm
     :cmpr #'l-cmpr
     :basis #'(lambda (n)
                      (mapcar #'dlop-int-ext (funcall (delta-n-basis dmns) n)))
     :bsgn '(0)
     :intr-dffr
         #'(lambda (degr gmsm)
            (make-cmbn
             :degr (1- degr)
             :list (do ((rslt +empty-list+
                              (cons (cons sign
                                          (append
                                           (subseq gmsm 0 nark)
                                           (subseq gmsm (1+ nark))))
                                    rslt))
                        (sign 1 (- sign))
                        (nark 0 (1+ nark)))
                       ((> nark degr) rslt))))
     :strt :gnrt
     :orgn `(Effective version of C_* delta ,dmns)))

In [ ]:
(def delta3 (cdelta 3))

In [ ]:
(basis delta3 0)

In [ ]:
(basis delta3 1)

In [ ]:
(basis delta3 2)

In [ ]:
(basis delta3 3)

In [ ]:
(basis delta3 4)

In [ ]:
(def bic (bicone (make-rdct 3 2) (make-rdct 4 2)))

In [ ]:
(basis bic 0)

In [ ]:
(basis bic 1)

In [ ]:
(basis bic 4)

In [ ]:
(? bic (cmbn 2 3 (bcnb '(0 1 3)) 4 (bcnc '(0 1 2 3)) 5 (bcnd '(0 1 4))))

In [ ]:
(? bic (? bic (cmbn 2 3 (bcnb '(0 1 3)) 4 (bcnc '(0 1 2 3)) 5 (bcnd '(0 1 4)))))

## 2.7 Composition of homotopy equivalences
#### Example

In [ ]:
(cat-init)

In [ ]:
(def c (build-chcm
            :cmpr #'s-cmpr
            :basis #'(lambda (dmns) (declare (ignore dmns)) '(a))
            :bsgn 'a
            :intr-dffr #'zero-intr-dffr
            :strt :cmbn
            :orgn '(c)))

In [ ]:
(def h1 (trivial-hmeq c))

In [ ]:
(def h2 (cmps h1 h1))

In [ ]:
(pre-check-rdct (lrdct h2))

In [ ]:
(setf *tc* (cmbn 3 1 (bcnb 'a) 10 (bcnc 'a) 100 (bcnd 'a)))

In [ ]:
(setf *bc* (cmbn 3 1 'a))

In [ ]:
(check-rdct-no-wait)

In [ ]:
(pre-check-rdct (rrdct h2))

In [ ]:
(check-rdct-no-wait)

In [ ]:
(def h3 (cmps h2 h2))

In [ ]:
(setf *tc* (cmbn 3 1 (bcnb (bcnb 'a)) 10 (bcnb (bcnc 'a)) 100 (bcnb (bcnd 'a))
                 1000 (bcnc 'a) 10000 (bcnd (bcnb 'a)) 5234 (bcnd (bcnc 'a))
                 223 (bcnd (bcnd 'a))))

In [ ]:
(pre-check-rdct (lrdct h3))

In [ ]:
(check-rdct-no-wait)

In [ ]:
(pre-check-rdct (rrdct h3))

In [ ]:
(check-rdct-no-wait)